# Data Preprocessing

## Import libraries

In [186]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
import random
import pycountry

## Read the data

In [110]:
df = pd.read_csv('salaries.csv')
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Business Intelligence Engineer,202800,USD,202800,US,0,US,L
1,2023,SE,FT,Business Intelligence Engineer,104300,USD,104300,US,0,US,L
2,2023,MI,FT,Data Manager,60000,GBP,73824,GB,0,GB,M
3,2023,MI,FT,Data Manager,55000,GBP,67672,GB,0,GB,M
4,2023,SE,FT,Data Scientist,290000,USD,290000,US,0,US,M


In [111]:
df.shape

(7974, 11)

## Data Exploration

* work_year: The year the salary was paid.
* experience_level: The experience level in the job during the year
```
EN: Entry-level / Junior
MI: Mid-level / Intermediate
SE: Senior-level / Expert
EX: Executive-level / Director
```
* employment_type: The type of employement for the role
```
PT: Part-time
FT: Full-time
CT: Contract
FL: Freelance
```
* job_title: The role worked in during the year.
* salary: The total gross salary amount paid.
* salary_currency: The currency of the salary paid as an ISO 4217 currency code.
* salary_in_usd: The salary in USD (FX rate divided by avg. USD rate of respective year via data from fxdata.foorilla.com).
* employee_residence: Employee's primary country of residence in during the work year as an ISO 3166 country code.
* remote_ratio: The overall amount of work done remotely
```
0: No remote work (less than 20%)
50: Partially remote/hybrid
100: Fully remote (more than 80%)
```
* company_location: The country of the employer's main office or contracting branch as an ISO 3166 country code.
* company_size: The average number of people that worked for the company during the year
```
S: less than 50 employees (small)
M: 50 to 250 employees (medium)
L: more than 250 employees (large)
```

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7974 entries, 0 to 7973
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           7974 non-null   int64 
 1   experience_level    7974 non-null   object
 2   employment_type     7974 non-null   object
 3   job_title           7974 non-null   object
 4   salary              7974 non-null   int64 
 5   salary_currency     7974 non-null   object
 6   salary_in_usd       7974 non-null   int64 
 7   employee_residence  7974 non-null   object
 8   remote_ratio        7974 non-null   int64 
 9   company_location    7974 non-null   object
 10  company_size        7974 non-null   object
dtypes: int64(4), object(7)
memory usage: 685.4+ KB


In [113]:
df.isna().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [114]:
df.describe(include='object')

,experience_level,employment_type,job_title,salary_currency,employee_residence,company_location,company_size
count,7974,7974,7974,7974,7974,7974,7974
unique,4,4,118,22,85,73,3
top,SE,FT,Data Engineer,USD,US,US,M
freq,5754,7932,1888,7216,6770,6817,7108


In [115]:
df.describe(exclude='object')

,work_year,salary,salary_in_usd,remote_ratio
count,7974.000000,7.974000e+03,7974.000000,7974.000000
mean,2022.710183,1.766936e+05,149369.311262,39.942313
std,0.563004,4.757161e+05,64057.210110,48.284731
min,2020.000000,1.400000e+04,15000.000000,0.000000
25%,2023.000000,1.060000e+05,105000.000000,0.000000
50%,2023.000000,1.449270e+05,142200.000000,0.000000
75%,2023.000000,1.900000e+05,185900.000000,100.000000
max,2023.000000,3.040000e+07,450000.000000,100.000000


## Dtypes checking

In [116]:
df.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

In [117]:
df['salary'] = df['salary'].astype(float)
df['salary_in_usd'] = df['salary_in_usd'].astype(float)

## Data cleaning

In [118]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Business Intelligence Engineer,202800.0,USD,202800.0,US,0,US,L
1,2023,SE,FT,Business Intelligence Engineer,104300.0,USD,104300.0,US,0,US,L
2,2023,MI,FT,Data Manager,60000.0,GBP,73824.0,GB,0,GB,M
3,2023,MI,FT,Data Manager,55000.0,GBP,67672.0,GB,0,GB,M
4,2023,SE,FT,Data Scientist,290000.0,USD,290000.0,US,0,US,M


In [119]:
df.nunique()

work_year                4
experience_level         4
employment_type          4
job_title              118
salary                1341
salary_currency         22
salary_in_usd         1616
employee_residence      85
remote_ratio             3
company_location        73
company_size             3
dtype: int64

In [120]:
# Map dicts for columns experience_level, employment_type and company_size
experience_level_map = {
    'EN': 'Entry-level',
    'MI': 'Mid-level',
    'SE': 'Senior-level',
    'EX': 'Executive-level'
}

employment_type_map = {
    'PT': 'Part-time',
    'FT': 'Full-time',
    'CT': 'Contract',
    'FL': 'Freelance'
}

remote_ratio_map = {
    0  : 'On-site',
    50 : 'Hybrid',
    100: 'Fully remote'
}

company_size_map = {
    'S': 'Small',
    'M': 'Medium',
    'L': 'Large'
}

In [122]:
df['experience_level'] = df['experience_level'].map(experience_level_map)
print(df['experience_level'].unique())

df['employment_type'] = df['employment_type'].map(employment_type_map)
print(df['employment_type'].unique())

df['remote_ratio'] = df['remote_ratio'].map(remote_ratio_map)
print(df['remote_ratio'].unique())

df['company_size'] = df['company_size'].map(company_size_map)
print(df['company_size'].unique())

['Senior-level' 'Mid-level' 'Entry-level' 'Executive-level']
['Full-time' 'Contract' 'Part-time' 'Freelance']
['On-site' 'Fully remote' 'Hybrid']
['Large' 'Medium' 'Small']


### Change iso2 codes to the real country names

In [123]:
df['company_location'].nunique()

73

In [124]:
df['employee_residence'].nunique()

85

In [127]:
df.groupby('company_location')['employee_residence'].nunique().sort_values()

company_location
AD     1
HN     1
HR     1
HU     1
IE     1
      ..
CA     5
AU     5
GB     8
DE    10
US    32
Name: employee_residence, Length: 73, dtype: int64

In [128]:
countries = {country.alpha_2 : country.name for country in pycountry.countries}

df['company_location'] = df['company_location'].map(countries)

df['employee_residence'] = df['employee_residence'].map(countries)

In [130]:
df.shape

(7974, 11)

In [131]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,Senior-level,Full-time,Business Intelligence Engineer,202800.0,USD,202800.0,United States,On-site,United States,Large
1,2023,Senior-level,Full-time,Business Intelligence Engineer,104300.0,USD,104300.0,United States,On-site,United States,Large
2,2023,Mid-level,Full-time,Data Manager,60000.0,GBP,73824.0,United Kingdom,On-site,United Kingdom,Medium
3,2023,Mid-level,Full-time,Data Manager,55000.0,GBP,67672.0,United Kingdom,On-site,United Kingdom,Medium
4,2023,Senior-level,Full-time,Data Scientist,290000.0,USD,290000.0,United States,On-site,United States,Medium


In [132]:
# Save final version to csv
df.to_csv('salaries_final.csv', index=False)

## Tests

**TEST #1**

In [219]:
df.groupby('remote_ratio')['salary_in_usd'].mean()

remote_ratio
Fully remote    144058.600260
Hybrid           82561.337963
On-site         155943.030549
Name: salary_in_usd, dtype: float64

### 1. Question: Is people working on a hybrid type of work get paid less than people working on Fully remote or On-site?

**Null Hypothesis**: People working on a hybrid type of work get paid the same as people working on Fully remote or On-site.

**Alternative Hypothesis**: People working on a hybrid type of work get paid less than people working on Fully remote or On-site.

Threshold (alpha) = 5%

Countries to consider = all

In [220]:
hybrid_salaries = df[df['remote_ratio'] == 'Hybrid']['salary_in_usd']
non_hybrid_salaries = df[df['remote_ratio'] != 'Hybrid']['salary_in_usd']

hybrid_salaries.shape, non_hybrid_salaries.shape

((216,), (7758,))

#### Imbalanced data

1. `Bootstrap` - Resample with replacement from both groups. This can help in estimating the sampling distribution and confidence intervals.

In [221]:
def bootstrap(data, num_samples=1000):
    sample_means = []
    for _ in range(num_samples):
        sample = data.sample(len(data), replace=True)
        sample_means.append(np.mean(sample))
    return sample_means

hybrid_bootstrap_means = bootstrap(hybrid_salaries)
non_hybrid_bootstrap_means = bootstrap(non_hybrid_salaries)

In [222]:
# Perform t-test
statistic, p_value = stats.ttest_ind(hybrid_bootstrap_means, non_hybrid_bootstrap_means)

# Define significance level (alpha)
alpha = 0.05

# Print results
print(f'Test Statistic: {statistic}')
print(f'P-Value: {p_value}')

# Compare p-value with significance level
if p_value < alpha:
    print("Reject null hypothesis: There is a significant difference in salaries.")
else:
    print("Fail to reject null hypothesis: There is no significant difference in salaries.")

Test Statistic: -530.7556563522479
P-Value: 0.0
Reject null hypothesis: There is a significant difference in salaries.


2. `stats.mannwhitneyu` is a non-parametric test that can be used to compare two independent samples.

In [223]:
# Perform Mann-Whitney U Test
U_statistic, p_value = stats.mannwhitneyu(hybrid_salaries, non_hybrid_salaries, alternative='less')

# Define significance level (alpha)
alpha = 0.05

# Print results
print(f'U Statistic: {U_statistic}')
print(f'P-Value: {p_value}')

# Compare p-value with significance level
if p_value < alpha:
    print("Reject null hypothesis: There is a significant difference in salaries.")
else:
    print("Fail to reject null hypothesis: There is no significant difference in salaries.")


U Statistic: 294961.0
P-Value: 8.191832740925051e-60
Reject null hypothesis: There is a significant difference in salaries.


#### The result

**Reject null hypothesis: There is a significant difference in salaries.**

**TEST #2**

In [230]:
df[df['company_location'] != df['employee_residence']].salary_in_usd.mean()

77621.66386554622

In [231]:
df[df['company_location'] == df['employee_residence']].salary_in_usd.mean()

150456.25843411838

### 2. Question: Do residents of other countries receive lower wages than citizens of the company's country?

**Null Hypothesis**: Residents of other countries do not receive lower wages than citizens of the company's country.

**Alternative Hypothesis**: Residents of other countries receive lower wages than citizens of the company's country.

Threshold (alpha) = 5%

Countries to consider = all

In [245]:
residents = df[df['company_location'] == df['employee_residence']]['salary_in_usd']
non_residents = df[df['company_location'] != df['employee_residence']]['salary_in_usd']

residents.shape, non_residents.shape

((7855,), (119,))

#### Imbalanced data

In [251]:
# Perform Mann-Whitney U Test
U_statistic, p_value = stats.mannwhitneyu(residents, non_residents, alternative='less')

# Define significance level (alpha)
alpha = 0.05

# Print results
print(f'U Statistic: {U_statistic}')
print(f'P-Value: {p_value}')

# Compare p-value with significance level
if p_value < alpha:
    print("Reject null hypothesis: Residents of other countries receive lower wages than citizens of the company's country.")
else:
    print("Fail to reject null hypothesis: Residents of other countries do not receive lower wages than citizens of the company's country.")

U Statistic: 777603.5
P-Value: 1.0
Fail to reject null hypothesis: Residents of other countries do not receive lower wages than citizens of the company's country.


In [252]:
# Perform Mann-Whitney U Test
U_statistic, p_value = stats.ttest_ind(bootstrap(residents), bootstrap(non_residents))

# Define significance level (alpha)
alpha = 0.05

# Print results
print(f'U Statistic: {U_statistic}')
print(f'P-Value: {p_value}')

# Compare p-value with significance level
if p_value < alpha:
    print("Reject null hypothesis: Residents of other countries receive lower wages than citizens of the company's country.")
else:
    print("Fail to reject null hypothesis: Residents of other countries do not receive lower wages than citizens of the company's country.")

U Statistic: 473.59083987352926
P-Value: 0.0
Reject null hypothesis: Residents of other countries receive lower wages than citizens of the company's country.
